In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from session_handler import *
import json
from datetime import datetime,date,timezone
import pytz
import urllib.parse
from klse_scrapper import *
from utils import *
import os
from workalendar.asia import Malaysia
from data_utils import *
from counter import Counter

In [2]:
sgtz=pytz.timezone('Asia/Singapore')

In [3]:
def crawl_time_sales(session,event):
    date_=datetime.fromtimestamp(event['to']).strftime("%Y%m%d")
    URL = 'http://www.shareinvestor.com/prices/time_and_sales_f.html'
    params={'date':date_,'counter':event['counter']+'.MY','page':-1}
    params=urllib.parse.urlencode(params)
    URL=URL+'?'+params
    page = session.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows=soup.find('table',{'id':"sic_timeAndSalesTable"}).findAll('tr')
    lists=[]
    for i in range(2,len(rows)):
        raw_vals=rows[i].findAll('td')
        time_=raw_vals[0].text
        type_=raw_vals[1].text
        last_done_=float(raw_vals[2].text)
        price_chg_=raw_vals[3].text
        vol_chg_=int(raw_vals[4].text.replace(',',''))
        vol_total_=int(raw_vals[5].text.replace(',',''))
        row={'time':date_+' '+time_,'type':type_,'last_done':last_done_,'price_chg':price_chg_,'vol_chg':vol_chg_,'vol_total':vol_total_}
        lists.append(row)
    df=pd.DataFrame(lists).set_index('time')
    df.to_csv('test.csv')

In [4]:
def crawl_quote_movements(session,event,live=False):
    date_=datetime.fromtimestamp(event['to'])
    date_=date_.strftime("%Y-%m-%d")
    empty_chars='\xa0'     
    URL = 'http://www.shareinvestor.com/prices/quote_movements_f.html'
    if live:
        pageno=1
    else:
        pageno=-1
    params={'date':date_,'counter':event['counter']+'.MY','page':pageno}
    params=urllib.parse.urlencode(params)
    URL=URL+'?'+params
    page = session.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows=soup.find('table',{'id':"sic_quoteMovementTable"}).findAll('tr')
    lists=[]
    for i in range(2,len(rows)):
        raw_vals=rows[i].findAll('td')
        if(len(raw_vals)>=7):
            time_=raw_vals[0].text
            bq_vol_ = None if raw_vals[1].text==empty_chars else raw_vals[1].text
            try:
                bq_price_=float(raw_vals[2].text)
            except Exception as e:
                bq_price_=None
            ld_vol=None if raw_vals[3].text==empty_chars else float(raw_vals[3].text.replace(',',''))
            try:
                ld_price=float(raw_vals[4].text)
            except Exception as e: 
                ld_price=None
            type_=None if raw_vals[5].text==empty_chars else raw_vals[5].text
            sq_vol_=None if raw_vals[6].text==empty_chars else raw_vals[6].text
            try:
                sq_price_=float(raw_vals[7].text)
            except Exception as e:
                sq_price_=None
            row={'time':date_+' '+time_,'buy_queue_vol':bq_vol_,'buy_queue_price':bq_price_,'last_done_vol':ld_vol,'last_done_price':ld_price,'type':type_,'sell_queue_vol':sq_vol_,'sell_queue_price':sq_price_}
            lists.append(row)
    df=pd.DataFrame(lists).set_index('time')
    return df

In [3]:
generate_session()
event={"counter":"5199",'page':1,'date':"2020-07-24"}
session=read_session()
URL = 'http://www.shareinvestor.com/prices/quote_movements_f.html'
params={'date':event['date'],'counter':event['counter']+'.MY','page':event['page']}
params=urllib.parse.urlencode(params)
URL=URL+'?'+params
print(URL)
page = session.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup)

http://www.shareinvestor.com/prices/quote_movements_f.html?date=2020-07-24&counter=5199.MY&page=1
<ul class="sic_toolBar">
<li class="sic_text">Page: <input class="sic_cur_page_input" type="text" value="1"/>/<span class="sic_total_page">26</span></li><li class="disabled sic_previous" title="Previous Page">Previous Page</li><li class="sic_next" title="Next Page">Next Page</li><li class="sic_all" title="View all pages">All</li>
<li class="sic_separator">-</li>
<li class="sic_print" title="Print">Print</li>
<li class="sic_text"><input class="sic_records_per_page_input sic_records_per_page {'section' : 'quote_movements'}" type="text" value="150">Rows/Page</input></li>
</ul>
<div class="sic_quoteContainer" id="sic_stockQuote">
<div class="mainInner">
<div class="subInner">
<form action="javascript:void(0);" id="sic_stockQuote_companyPicker">
<ul>
<li class="sic_company">
<label for="counter">Counter Name:</label>
<select id="market" name="market"><option class="{access_allowed : false}" val

In [6]:
def crawl_quote_movements(session,event,live=False):
    date_=datetime.fromtimestamp(event['to'])
    date_=date_.strftime("%Y-%m-%d")    
    URL = 'http://www.shareinvestor.com/prices/quote_movements_f.html'
    if live:
        pageno=1
    else:
        pageno=-1
    params={'date':date_,'counter':event['counter']+'.MY','page':pageno}
    params=urllib.parse.urlencode(params)
    URL=URL+'?'+params
    page = session.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    table=soup.find('table',{'id':"sic_quoteMovementTable"})
    #use pandas read_html as a more straightforward to parse html table
    df = pd.read_html(str(table))[0]
    new_cols = list()
    for col in df.columns.values:
        if col[0] == col[1]: new_cols.append(col[0].lower())
        else: new_cols.append(' '.join(col).lower().replace(' ','_'))
    df.columns = new_cols
    tmp=df['buy_queue_vol'].str.split(' ',expand=True)
    df['buy_vol_chg']=tmp[0].str.extract("\((.*?)\)").astype(float)
    df['buy_queue_vol']=tmp[1].str.replace(',','').astype(float)
    tmp=df['sell_queue_vol'].str.split(' ',expand=True)
    df['sell_vol_chg']=tmp[0].str.extract("\((.*?)\)").astype(float)
    df['sell_queue_vol']=tmp[1].str.replace(',','').astype(float)
    df['time']=pd.to_datetime(df['time'])
    return df

Time      Buy Queue       Last Done             Type Sell Queue      
         Time            Vol Price       Vol Price       Type        Vol Price
0    16:59:51   (+42) 32,482  0.61       NaN   NaN        NaN        NaN   NaN
1    16:59:34  (+200) 32,440  0.61       NaN   NaN        NaN        NaN   NaN
2    16:58:59            NaN   NaN     185.0  0.61  Sell Down        NaN   NaN
3    16:58:59            NaN   NaN      15.0  0.61  Sell Down        NaN   NaN
4    16:58:59  (-185) 32,240  0.61       NaN   NaN        NaN        NaN   NaN
..        ...            ...   ...       ...   ...        ...        ...   ...
145  16:50:07            NaN   NaN      50.0  0.61  Sell Down        NaN   NaN
146  16:50:07            NaN   NaN     300.0  0.61  Sell Down        NaN   NaN
147  16:50:07            NaN   NaN     300.0  0.61  Sell Down        NaN   NaN
148  16:50:07            NaN   NaN     500.0  0.61  Sell Down        NaN   NaN
149  16:50:07            NaN   NaN     100.0  0.61  Sell Down        NaN   NaN

[150 rows x 8 columns]

In [97]:
df['Buy Queue Vol'].str.extract("(\d+) (\d+)")

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
145,NaN,NaN
146,NaN,NaN
147,NaN,NaN
148,NaN,NaN


In [5]:
def live_quote_movements(session,event):
    df_=crawl_quote_movements(session,event,True)
    date_=event['to']
    df_=df_.reset_index()
    df_['time']=pd.to_datetime(df_['time'])
    for i,row in df_.iterrows():
        if(row['time'].replace(tzinfo=sgtz).timestamp()<date_):
            break    
    return df_[0:i-1]

In [6]:
def mine_quote_movements(session,board,category,from_,to_):
    df=get_board_category_listings(board,category)
    lists_=df[['symbol','code','cat']].to_dict('records')
    events=build_event_lists(lists_=lists_,from_=from_,to_=to_)
    for event in events:
        events_=distribute_requests(event)
        for event_ in events_:
            try:
                cal = Malaysia()
                date_=datetime.fromtimestamp(event_['to'])
                if(cal.is_working_day(date(date_.year,date_.month,date_.day))):
                    date_=date_.strftime("%Y-%m-%d")
                    dir_='history/quote_movements/'
                    dir_=os.path.join(dir_,event_['category'])
                    name=event_['symbol']+'_'+event_['counter']
                    dir_=os.path.join(dir_,name)
                    if not os.path.exists(dir_): os.makedirs(dir_)
                    save_path=os.path.join(dir_,name)+'_'+date_+'.csv'
                    #if csv exists, return dataframe from existing csv
                    if os.path.exists(save_path): 
                        print('CSV exists. Not going to crawl.')
                    else:
                        df=crawl_quote_movements(session,event_)
                        df.to_csv(save_path)
            except Exception as e: print(e)

In [1]:
event={'to':1591952280,'counter':'2127'}
%time df=live_quote_movements(session,event)
df = split_buy_sell_queue(df)
df=df.iloc[::-1]
df.groupby('type').agg({'last_done_price':'last'})
df.groupby('sell_queue_price').agg({'sell_queue_vol':'last','sell_vol_chg':'sum'})
df.groupby('buy_queue_price').agg({'buy_queue_vol':'last','buy_vol_chg':'sum'})
df

NameError: name 'live_quote_movements' is not defined

NameError: name 'split_buy_sell_queue' is not defined

In [10]:
generate_session()
session=read_session()
%time mine_quote_movements(session=session,board='Main Market',category='Energy',from_=1585702800,to_=1588986000)

CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
CSV exists. Not going to crawl.
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in the columns"
"None of ['time'] are in

In [2]:
dufu=Counter('DUFU','7183',scheduler_on=False)
#dufu.refresh_volume()
dufu.get_day_volume()
#dufu.detect_shark()

DUFU


C:\ProgramData\Anaconda3\envs\fastai\lib\site-packages\workalendar\core.py:719: UserWarning: Please take note that, due to arbitrary decisions, this Islamic calendar computation may be wrong.
  warnings.warn('Please take note that, due to arbitrary decisions, '


https://www.klsescreener.com/v2/trading_view/history?symbol=7183&resolution=D&from=1595538000.0&to=1595581200.0


In [10]:
dufu.buffer_temp.buy_vol_chg[dufu.buffer_temp.buy_vol_chg.notna()]

5        10.0
7      -200.0
9       -50.0
15     -200.0
16      -38.0
        ...  
518    -300.0
519     -50.0
522     100.0
523      50.0
526    1000.0
Name: buy_vol_chg, Length: 197, dtype: float64